In [ ]:
# %pip install beautifulsoup4
# %pip install requests

In [ ]:
# %pip install urllib3==1.26.7

In [ ]:
import requests

url = 'https://rumah.trovit.co.id/index.php/cod.search_adwords_homes/ppc_landing_type.3/type.1/what_d.Nusa%20Tenggara%20Barat/sug.0/isUserSearch.1/origin.1/order_by.relevance/region.Nusa%20Tenggara%20Barat/property_type.Rumah/'  # web target
response = requests.get(url)

if response.status_code == 200:
    content = response.content


In [ ]:
%pip install selenium

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(content, 'html.parser')

title = soup.title
print(title.text)


In [ ]:
links = soup.find_all('a')

for link in links:
    print(link['href'])

In [ ]:
import requests

response = requests.get(url)

if response.status_code == 200:
    content = response.content

In [ ]:
inner_soup = BeautifulSoup(content, 'html.parser')

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

# URL halaman dashboard
dashboard_url = 'https://rumah.trovit.co.id/index.php/cod.search_adwords_homes/ppc_landing_type.3/type.1/what_d.Nusa%20Tenggara%20Barat/sug.0/isUserSearch.1/origin.1/order_by.relevance/region.Nusa%20Tenggara%20Barat/property_type.Rumah/'
# Ganti dengan URL halaman dashboardmu

# Mengirim permintaan HTTP ke halaman dashboard
dashboard_response = requests.get(dashboard_url)

if dashboard_response.status_code == 200:
    dashboard_content = dashboard_response.content
    dashboard_soup = BeautifulSoup(dashboard_content, 'html.parser')

    # Mengambil daftar link menuju halaman detail properti
    property_links = []
    table_rows = dashboard_soup.find_all('tr')
    for row in table_rows:
        link = row.find('a')['href']
        property_links.append(link)

    # Membuat file CSV untuk menyimpan data
    csv_file = open('property_data.csv', 'w', newline='')
    writer = csv.writer(csv_file)
    
    # Menulis header ke dalam file CSV
    header = ['K. Tidur', 'K. Mandi', 'L. Tanah', 'L. Bangunan', 'Tipe Properti', 'Sertifikat', 'Daya Listrik', 'ID Iklan']
    writer.writerow(header)
    
    # Melakukan scraping dari halaman detail properti
    for link in property_links:
        property_url = 'https://www.example.com' + link
        # Ganti dengan URL domainmu
        
        # Mengirim permintaan HTTP ke halaman detail properti
        property_response = requests.get(property_url)
        
        if property_response.status_code == 200:
            property_content = property_response.content
            property_soup = BeautifulSoup(property_content, 'html.parser')

            # Mengambil spesifikasi rumah dari halaman detail properti
            specification_rows = property_soup.find_all('div', class_='ui-listing-specification__table--row')
            specification_data = {}

            for row in specification_rows:
                label = row.find('p', class_='ui-listing-specification__table--label').text
                value = row.find('p', class_='ui-listing-specification__table--value').text
                specification_data[label] = value
            
            # Menulis data spesifikasi rumah ke dalam file CSV
            row_data = [specification_data.get(header, '') for header in header]
            writer.writerow(row_data)
    
    # Menutup file CSV setelah selesai
    csv_file.close()

    print("Web scraping selesai. Data telah disimpan dalam file 'property_data.csv'.")
else:
    print("Gagal mengakses halaman dashboard.")


In [ ]:
import csv
import requests
from bs4 import BeautifulSoup

# URL halaman dashboard
dashboard_url = 'https://rumah.trovit.co.id/index.php/cod.search_adwords_homes/ppc_landing_type.3/type.1/what_d.Nusa%20Tenggara%20Barat/sug.0/isUserSearch.1/origin.1/order_by.relevance/region.Nusa%20Tenggara%20Barat/property_type.Rumah/'
# Ganti dengan URL halaman dashboardmu

# Mengirim permintaan HTTP ke halaman dashboard
response = requests.get(dashboard_url)

if response.status_code == 200:
    dashboard_content = response.content
    dashboard_soup = BeautifulSoup(dashboard_content, 'html.parser')

    # Mendapatkan harga dari halaman dashboard
    harga_element = dashboard_soup.find('div', class_='item-price')
    harga = harga_element.find('span', class_='actual-price').text.strip()

    # Mendapatkan link detail properti dari halaman dashboard
    links = dashboard_soup.find_all('a', class_='rd-link image-link')

    # Inisialisasi list untuk menyimpan data properti
    data_properti = []

    # Loop melalui setiap link detail properti
    for link in links:
        detail_url = link['href']
        detail_response = requests.get(detail_url)

        if detail_response.status_code == 200:
            detail_content = detail_response.content
            detail_soup = BeautifulSoup(detail_content, 'html.parser')

            # Mendapatkan lokasi dari halaman detail properti
            lokasi_element = detail_soup.find('div', class_='r123-listing-summary__header-container-address')
            lokasi = lokasi_element.string.strip() if lokasi_element else ''

            # Mendapatkan spesifikasi properti dari halaman detail properti
            spesifikasi_element = detail_soup.find('div', class_='ui-listing-specification__table')
            rows = spesifikasi_element.find_all('div', class_='ui-listing-specification__table--row')

            # Membuat dictionary untuk menyimpan data spesifikasi properti
            properti = {}

            for row in rows:
                label = row.find('p', class_='ui-listing-specification__table--label').text.strip()
                value = row.find('p', class_='ui-listing-specification__table--value').text.strip()
                properti[label] = value

            # Menambahkan harga dan lokasi ke dictionary properti
            properti['Harga'] = harga
            properti['Lokasi'] = lokasi

            # Menambahkan dictionary properti ke list data_properti
            data_properti.append(properti)

    # Menyimpan data properti ke dalam file CSV
    header = ['K. Tidur', 'K. Mandi', 'L. Tanah', 'L. Bangunan', 'Tipe Properti', 'Sertifikat', 'Daya Listrik', 'ID Iklan', 'Harga', 'Lokasi']
    with open('data_properti.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=header)
        writer.writeheader()
        writer.writerows(data_properti)

    print('Web scraping selesai. Data properti telah disimpan dalam file data_properti.csv')
else:
    print('Gagal mengakses halaman dashboard')


In [2]:
import csv
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# URL halaman dashboard
dashboard_url = 'https://rumah.trovit.co.id/index.php/cod.search_adwords_homes/ppc_landing_type.3/type.1/what_d.Nusa%20Tenggara%20Barat/sug.0/isUserSearch.1/origin.1/order_by.relevance/region.Nusa%20Tenggara%20Barat/property_type.Rumah/'
# Ganti dengan URL halaman dashboardmu

# Mengirim permintaan HTTP ke halaman dashboard
response = requests.get(dashboard_url)

if response.status_code == 200:
    dashboard_content = response.content
    dashboard_soup = BeautifulSoup(dashboard_content, 'html.parser')

    # Mendapatkan harga dari halaman dashboard
    harga_element = dashboard_soup.find('div', class_='item-price')
    harga = harga_element.find('span', class_='actual-price').text.strip()

    # Mendapatkan link detail properti dari halaman dashboard
    links = dashboard_soup.find_all('a', class_='rd-link image-link')

    # Inisialisasi list untuk menyimpan data properti
    data_properti = []

    # Inisialisasi WebDriver Selenium
    driver = webdriver.Chrome()  # Ganti dengan driver sesuai preferensi (misal: Firefox, Safari)

    # Loop melalui setiap link detail properti
    for link in links:
        detail_url = link['href']
        driver.get(detail_url)

        try:
            # Cari elemen teks link dan klik
            # link_element = driver.find_element(By.XPATH, '//div[contains(text(), "Menampilkan lebih banyak")]')
            # driver.execute_script("arguments[0].click();", link_element)
            # Cari elemen teks link dan klik

            link_element = driver.find_element(By.XPATH, '//div[contains(text(), "Menampilkan lebih banyak")]')
            driver.execute_script("arguments[0].click();", link_element)

            # Tunggu beberapa detik
            time.sleep(5)  # Ubah angka 5 menjadi jumlah detik yang Anda anggap perlu

            # Ambil konten HTML setelah modal view muncul
            detail_content = driver.page_source

            # Tunggu hingga modal view muncul
            try:
                WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, 'ui-listing-specification__table--row')))
            
                # Ambil konten HTML setelah modal view muncul
                detail_content = driver.page_source
            except NoSuchElementException:
                # Skip data properti jika tidak ditemukan elemen "Menampilkan lebih banyak"
                continue

        except NoSuchElementException:
            # Skip data properti jika tidak ditemukan elemen "Menampilkan lebih banyak"
            continue



        # Buat objek BeautifulSoup baru dari konten HTML yang diperoleh
        detail_soup = BeautifulSoup(detail_content, 'html.parser')

        # Mendapatkan lokasi dari halaman detail properti
        lokasi_element = detail_soup.find('div', class_='r123-listing-summary__header-container-address')
        lokasi = lokasi_element.text.strip() if lokasi_element else ''

        # Mendapatkan spesifikasi properti dari halaman detail properti
        # spesifikasi_element = detail_soup.find('div', class_='ui-listing-specification__table')
        # rows = spesifikasi_element.find_all('div', class_='ui-listing-specification__table--row')
        # Mendapatkan spesifikasi properti dari halaman detail properti
        spesifikasi_element = detail_soup.find('div', class_='ui-listing-specification__table')
        if spesifikasi_element is None:
            continue

        rows = spesifikasi_element.find_all('div', class_='ui-listing-specification__table--row')


        # Membuat dictionary untuk menyimpan data spesifikasi properti
        properti = {}

        for row in rows:
            label = row.find('p', class_='ui-listing-specification__table--label').text.strip()
            value = row.find('p', class_='ui-listing-specification__table--value').text.strip()
            properti[label] = value

        # Menambahkan harga dan lokasi ke dictionary properti
        properti['Harga'] = harga
        properti['Lokasi'] = lokasi

        # Menambahkan dictionary properti ke list data_properti
        data_properti.append(properti)

    driver.quit()

    # Menyimpan data properti ke dalam file CSV
    header = ['K. Tidur', 'K. Mandi', 'L. Tanah', 'L. Bangunan', 'Tipe Properti', 'Sertifikat', 'Daya Listrik', 'ID Iklan', 'Harga', 'Lokasi']
    with open('data_properti.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=header)
        writer.writeheader()
        writer.writerows(data_properti)

    print('Web scraping selesai. Data properti telah disimpan dalam file data_properti.csv')
else:
    print('Gagal mengakses halaman dashboard')


TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x0000000102abf8ac chromedriver + 4257964
1   chromedriver                        0x0000000102ab7f40 chromedriver + 4226880
2   chromedriver                        0x00000001026f49d4 chromedriver + 281044
3   chromedriver                        0x000000010272fa34 chromedriver + 522804
4   chromedriver                        0x00000001027667e4 chromedriver + 747492
5   chromedriver                        0x000000010272398c chromedriver + 473484
6   chromedriver                        0x000000010272498c chromedriver + 477580
7   chromedriver                        0x0000000102a7e900 chromedriver + 3991808
8   chromedriver                        0x0000000102a82354 chromedriver + 4006740
9   chromedriver                        0x0000000102a82940 chromedriver + 4008256
10  chromedriver                        0x0000000102a8833c chromedriver + 4031292
11  chromedriver                        0x0000000102a82f34 chro